# Visualisation

In [2]:
import pandas as pd

df = pd.read_csv('../data/tx_statewide_2020_04_01-002_clean.csv')
df

,"route: 0040, milepost: 156",35.22798156738281,-100.38148498535156,Wheeler County,4,0,0.0,0.0.1,0.1,2010-04-27 15:10:00
0,"route: 0083, milepost: 582",29.227333,-99.793602,Uvalde County,0,1,0.0,0.0,2003,2010-04-27 15:10:00
1,"route: 0010, milepost: 508",30.070118,-99.102036,Kerr County,0,1,0.0,0.0,2005,2010-04-27 15:10:00
2,"route: 0707, milepost: 304",32.352283,-99.784698,Taylor County,0,1,0.0,0.0,0,2010-04-27 15:14:00
3,"route: 1395, milepost: 266",32.736950,-95.896782,Van Zandt County,0,1,0.0,0.0,2005,2010-04-27 15:14:00
4,"route: 0035W, milepost: 11",32.201431,-97.144402,Hill County,0,0,0.0,0.0,2010,2010-04-27 15:14:00
...,...,...,...,...,...,...,...,...,...,...
39505566,"route: 0010, milepost: 416",NaN,NaN,Sutton County,0,1,0.0,0.0,0,2006-12-28 16:05:00
39505567,"route: 0035, milepost: 488",NaN,NaN,Brazoria County,0,0,0.0,0.0,0,2006-12-28 16:05:00
39505568,"route: 0021, milepost: 802",NaN,NaN,Nacogdoches County,0,0,0.0,0.0,0,2006-12-28 16:05:00
39505569,"route: 0080, milepost: 476",NaN,NaN,Caldwell County,0,1,0.0,0.0,0,2006-12-28 16:05:00


[Macro](#Macro)
- Median number of tickets per day.
- Plot the number of tickets per hour of the day/Does time of day affect crime.
- Plot the distribution of tickets by county or location.
- Plot the yearly evolution of the number of tickets.
- Investigate if there's a correlation between the number of tickets and specific events or holidays.

[Disparity](#Disparity)
- Gender distribution of tickets. (take into account the proportion of population)
- Plot the correlation between men and women getting a ticket for speeding.
- Explore if certain vehicle types or colors are more likely to get a ticket.
- How predictable are the stop rates, is there a time and place where you are more likely to get a ticket ?
- Address the concern about jurisdictions using civil forfeiture as a funding mechanism rather than to properly fight drug trafficking.